<a href="https://colab.research.google.com/github/GregoryG3/Thesis/blob/main/INITIALIZATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INITIALIZATION

# Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
from os.path import join

from matplotlib import pyplot as plt
import seaborn as sns

# import random
# from datetime import date, timedelta
# import os


# from transformers import (AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification,
#                           BertTokenizer, AdamW, Trainer, TrainingArguments, XLNetTokenizer, XLNetForSequenceClassification,
#                           RobertaTokenizer, RobertaForSequenceClassification, AutoModelForSeq2SeqLM, pipeline)
# import torch
# from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from tqdm import tqdm

# from sklearn.metrics import (f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,
                            #  accuracy_score, precision_score, recall_score)
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# import string
# import nltk
# from nltk.util import ngrams
# from nltk.corpus import stopwords
# from collections import Counter
# from bs4 import BeautifulSoup
# import re


# from matplotlib.lines import Line2D
# from wordcloud import WordCloud
# # from PIL import Image

# Load data

In [2]:
drive.mount('/content/drive')
PROJECT_DIR = "/content/drive/MyDrive/Thesis/Data"
FEEDBACK_DIR = join(PROJECT_DIR, "Feedback data")

Mounted at /content/drive


In [ ]:
df_predict = pd.read_csv(join(FEEDBACK_DIR, "sr_ab_ha_for_prediction.csv"), low_memory=False)


# FULL FEEDBACK - DESCRIPTIVE STATISTICS

In [ ]:
df_predict

,seller,date_left,buyer,comment,itemid,price,rating,category,global_id,dataset
0,4abff8b0b6,2012-03-22,none,Package was discrete & smell-proof. Quick deli...,0001a78fac,6.71,Positive,Cannabis,sr1,sr
1,4abff8b0b6,2012-05-11,none,"Perfect transaction......fast, nicely packaged...",0001a78fac,6.71,Positive,Cannabis,sr2,sr
2,1615f9459a,2012-06-11,none,Fast delivery. Creative stealth package. Thank...,000aa84cce,33.56,Positive,"Steroids, PEDs",sr3,sr
3,3affb14050,2012-07-23,none,Thanks kitty.. Got it! Will try and update ove...,000b934049,13.17,Positive,Pills,sr4,sr
4,3affb14050,2012-06-16,none,didnt arrive had to do refund but seller was v...,000b934049,13.17,Neutral,Pills,sr5,sr
...,...,...,...,...,...,...,...,...,...,...
1906108,GGMCCLOUD1TEAM,2016-12-08,u***e,WILL BUY FROM AGAIN!!!,55366,12.00,Positive,Fraud Related,ha51322,ha
1906109,GGMCCLOUD1TEAM,2016-12-09,u***e,I keep buying because I keep winning! LETS GO!!,55366,12.00,Positive,Fraud Related,ha51323,ha
1906110,GGMCCLOUD1TEAM,2016-12-08,s***r,thnxx and fast ship bro,55366,12.00,Positive,Fraud Related,ha51325,ha
1906111,OhishNiceScript,2016-12-12,J***2,Took a chance on a new vendor and wasn't disap...,55675,90.00,Positive,Drugs,ha51331,ha


In [ ]:
df_predict['comment'].isna().value_counts()

comment
False    1905730
True         383
Name: count, dtype: int64

In [ ]:
df_predict[df_predict['comment'].isna() == True]

,seller,date_left,buyer,comment,itemid,price,rating,category,global_id,dataset
29905,3ee749d068,2012-04-16,none,NaN,2ab6d1eb53,21.07,Neutral,Weed,sr31010,sr
105893,4fc2568952,2012-05-20,none,NaN,9322f5faa6,0.00,Positive,Opioids,sr109843,sr
178773,HumboldtFarms,2016-09-27,i...n,NaN,29998,297.00,Positive,Cannabis & Hashish,ao968685,ab
185884,HumboldtFarms,2016-10-09,v...x,NaN,29982,42.00,Positive,Cannabis & Hashish,ao967186,ab
186922,HumboldtFarms,2016-01-07,k...4,NaN,67535,32.00,Positive,NaN,ao994299,ab
...,...,...,...,...,...,...,...,...,...,...
1864109,spectre,2016-11-28,b...y,NaN,16854,2.00,Positive,Legit Software,ao1739414,ab
1866965,Sn4pp,2016-08-08,m...6,NaN,190243,53.60,Positive,Opioids,ao1726944,ab
1866967,Sn4pp,2016-08-08,m...6,NaN,190239,99.00,Positive,Opioids,ao1726943,ab
1870747,StraightFlush,2016-03-18,m...y,NaN,115410,10.97,Positive,NaN,ao1785930,ab


In [ ]:
test = pd.DataFrame(df_predict['category'].value_counts())

In [ ]:
test

,count
category,
Cannabis & Hashish,324232
Stimulants,202837
Ecstasy,119929
Opioids,106740
Psychedelics,97169
...,...
Toys,1
A-PVP,1
Precursors,1


In [ ]:
df_predict[df_predict['category'].isna()== True]

,seller,date_left,buyer,comment,itemid,price,rating,category,global_id,dataset
178139,HumboldtFarms,2016-03-21,d...g,"5/5 everything was perfect. Great product, gr...",40256,75.0,Positive,NaN,ao988317,ab
178214,HumboldtFarms,2016-02-11,t...e,"Very nice stealth, amazing product, extremely ...",67535,32.0,Positive,NaN,ao991751,ab
178228,HumboldtFarms,2016-09-16,l...g,Fast shipping and great quality. Thanks Humbolt.,70404,800.0,Positive,NaN,ao970315,ab
178239,HumboldtFarms,2016-06-18,i...n,"took 6 days but well worth the wait, this stuf...",67546,242.0,Positive,NaN,ao979637,ab
178282,HumboldtFarms,2016-09-01,d...n,"Twice as long to pack and ship out, not like o...",70404,807.0,Positive,NaN,ao971902,ab
...,...,...,...,...,...,...,...,...,...,...
1880325,Telek0m,2017-02-22,n...m,"Original Medikamente, annehmbare Lieferzeit (3...",293365,3.2,Positive,NaN,ao1831950,ab
1880328,the-original-pharmacypowder,NaN,d...s,12 day no show with no response for the last c...,168444,10.0,Negative,NaN,ao2243180,ab
1880332,TheCEO,2017-05-04,d...2,Received 4dd. Was 2.1 grams short. A lot more ...,339283,410.0,Neutral,NaN,ao2210479,ab
1880358,weedim,2016-05-15,p...9,FE for my trusted supplier,150199,6.9,Positive,NaN,ao2117649,ab


In [ ]:
 df_predict['category'].isna().sum()

In [ ]:
 (df_predict['category'].isna().sum()/ df_predict['category'].sum())*100